In [ ]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec

from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

In [ ]:
from nmt_run import *

In [ ]:
last_epoch = check_model()

In [ ]:
t = L.Linear(5,5, nobias=True)

In [ ]:
# a dimensions: batch_size, sequence length, hidden units
a = Variable(np.random.randn(9,7,5).astype(np.float32))
b = Variable(np.random.randn(1,5).astype(np.float32))
a.shape, b.shape

In [ ]:
x = t(b)
# F.matmul(a,x, transb=True)

In [ ]:
# F.softmax(F.matmul(a,t(b),transb=True),axis=0)

In [ ]:
F.rollaxis(a,2).shape

### Get batch of data

In [ ]:
def get_full_length_batch(m_dict, x_key, y_key,
              utt_list, vocab_dict,
              max_enc, max_dec, cat_speech_path=''):

    batch_data = {'X':[], 'y':[]}
    for u in utt_list:
        # for speech data
        x_data_found = False
        if x_key == 'sp':
            utt_sp_path = os.path.join(cat_speech_path,
                                      "{0:s}.npy".format(u))
            if not os.path.exists(utt_sp_path):
                utt_sp_path = os.path.join(cat_speech_path,
                                           u.split('_',1)[0],
                                           "{0:s}.npy".format(u))
            if os.path.exists(utt_sp_path):
                x_data_found = True
                x_data = xp.load(utt_sp_path)
                print(x_data.shape)
                batch_data['X'].append(x_data)
            else:
                print("ERROR!! file not found: {0:s}".format(utt_sp_path))
        else:
            u_x_ids = [vocab_dict[x_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][x_key]]
            u_x_ids = xp.asarray(u_x_ids, dtype=xp.int32)
            x_data_found = True
            batch_data['X'].append(u_x_ids)
        #  add english labels

        if x_data_found:
            if type(m_dict[u][y_key]) == list:
                en_ids = [vocab_dict[y_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key]]
            else:
                # dev and test data have multiple translations
                # choose the first one for computing perplexity
                en_ids = [vocab_dict[y_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key][0]]
            u_y_ids = [GO_ID] + en_ids + [EOS_ID]
            if x_data_found == True:
                batch_data['y'].append(xp.asarray(u_y_ids, dtype=xp.int32))
    # end for all utterances in batch
    if len(batch_data['X']) > 0 and len(batch_data['y']) > 0:
        batch_data['X'] = F.pad_sequence(batch_data['X'], padding=PAD_ID)
        batch_data['y'] = F.pad_sequence(batch_data['y'], padding=PAD_ID)
    return batch_data

In [ ]:
key = 'fisher_train'

In [ ]:
b_dict=bucket_dict[key]
m_dict = map_dict[key]
vocab_dict=vocab_dict
batch_size=1
x_key=enc_key
y_key=dec_key
train=True
cat_speech_path=os.path.join(out_path, key)

b = num_b-1
# b = 9
bucket = b_dict['buckets'][b]
b_len = len(bucket)

In [ ]:
num_b, b_len

In [ ]:
utt_list = bucket[:batch_size]

In [ ]:
en_chars = [w.decode() for w in m_dict[utt_list[0]][y_key]]
en_ids = [str(vocab_dict[y_key]['w2i'].get(w, UNK_ID)) for w in m_dict[utt_list[0]][y_key]]

In [ ]:
print(" ".join(en_chars))
print(en_ids)

In [ ]:
# optimizer = optimizers.Adam()
# optimizer.setup(model)

In [ ]:
print("bucket size={0:d}".format(b_len))

for i in range(0,batch_size, batch_size):
    utt_list = bucket[i:i+batch_size]
    batch_data = get_batch(m_dict, x_key, y_key, utt_list, vocab_dict, ((b+1) * width_b), (80), 
                           cat_speech_path=cat_speech_path)
    
    print(i, batch_data['X'].shape, batch_data['y'].shape)
    
    p, loss = model.forward(batch_data['X'], batch_data['y'])
    
    model.cleargrads()
    
    loss.backward()
    # loss_1.backward()

    # model.addgrads(model_1)
    optimizer.update()
    
    print(loss)
    break

In [ ]:
import chainer.computational_graph as c

In [ ]:
comp_graph_name = log_train_fil_name.replace(".log", ".graph")

In [ ]:
# g = c.build_computational_graph([model.loss])

In [ ]:
# i = 0
# for node in g.nodes:
#     try:
#         print(node.label)
#     except:
#         print("label not found")

In [ ]:
g = c.build_computational_graph([model.loss])
with open(comp_graph_name, 'w') as o:
    o.write(g.dump())

In [ ]:
comp_graph_name

In [ ]:
model.L2_dec.W.W[:5,:5]

In [ ]:
model.L2_dec.W.W.data[0,0] = 0.01

In [ ]:
model.add_weight_noise(0.,0.125)

In [ ]:
model.L2_dec.W.W[:5,:5]

In [ ]:
model.L2_dec.W.update_enabled

In [ ]:
optimizer.setup(model)

In [ ]:
model.L2_dec.W

In [ ]:
model.L2_dec.W

In [ ]:
20000 * 64 / 138708

In [ ]:
xp.random.normal(0, .125, (5,5), dtype=xp.float32)

In [ ]:
model.rnn_enc

In [ ]:
model.L0_dec.W, model.L0_dec.W_r, model.L0_dec.W_z

In [ ]:
model.L0_dec.U, model.L0_dec.U_r, model.L0_dec.U_z

In [ ]:
model.L0_dec.U.W.shape, model.L0_dec.U.b.shape, model.L0_dec.U_r.W.shape, model.L0_dec.U_r.b.shape

In [ ]:
model.embed_dec.W.shape

In [ ]:
mu, sigma = 0, 0.125 # mean and standard deviation

for rnn_layer in model.rnn_enc + model.rnn_dec:
    print(rnn_layer)
    layer_shape = model[rnn_layer]["W"].W.shape
    print(layer_shape)
    

In [ ]:
reduce_dim = CNN_IN_DIM
cnn_out_dim = 0
for i, l in enumerate(cnn_filters):
    cnn_out_dim += l["out_channels"]
    print(reduce_dim, l["stride"][1])
    reduce_dim = math.ceil(reduce_dim / l["stride"][1])
    print(reduce_dim)
    print(l["out_channels"])


In [ ]:
cnn_out_dim

In [ ]:
CNN_IN_DIM

In [ ]:
cnn_filters[-1]["out_channels"]

In [ ]:
x = batch_data['X']

In [ ]:
x.shape

In [ ]:
F.expand_dims(x,2).shape

In [ ]:
model.cleargrads()

In [ ]:
loss.backward()

In [ ]:
if lstm1_or_gru0:
    print(model.L1_dec.lateral.W.grad)
else:
    print(model.L1_dec.W.W.grad)

In [ ]:
model.attn_Wa.W.grad

In [ ]:
model.out.W.grad

In [ ]:
if lstm1_or_gru0:
    print(model.L0_enc.lateral.W.grad)
else:
    print(model.L0_enc.W.W.grad)

In [ ]:
def plot_model_weights_grads(layer):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20, 10))
    plt.axis('off')
    weights = layer.data
    grads = layer.grad
    if gpuid >= 0:
        weights = cuda.cupy.asnumpy(weights[:,:])
        grads = cuda.cupy.asnumpy(grads[:,:])
    # plt.imshow(weights, interpolation='none')
    ax1 = sns.heatmap(weights, cmap="RdBu_r", ax=ax1, xticklabels=False, yticklabels=False)
    ax2 = sns.heatmap(grads, cmap="RdBu_r", ax=ax2, xticklabels=False, yticklabels=False)
    print(weights.shape)

In [ ]:
plot_model_weights_grads(model.L0_enc.W.W)

In [ ]:
plot_model_weights_grads(model.L1_dec.W.W)

In [ ]:
plot_model_weights_grads(model.out.W)

In [ ]:
xp.min(model.CNN_0.W.data)

In [ ]:
xp.nonzero(model.embed_dec.W.grad)

In [ ]:
x_key, y_key

In [ ]:
es_c_utt_lens = []
en_c_utt_lens = []
total_num = 0
for b in b_dict['buckets']:
    print(len(b), end=', ')
    print(max([len(m_dict[u]['es_w']) for u in b]), end=', ')
    print(max([len(m_dict[u]['es_c']) for u in b]), end=', ')
    print(max([len(m_dict[u]['es_c'])//6 for u in b]), end=', ')
    print(max([len(m_dict[u]['en_w']) for u in b]), end=', ')
    print(max([len(m_dict[u]['en_c']) for u in b]), end=', ')
    print(total_num, end=', ')
    print("")
    total_num += len(b)
    es_c_utt_lens.extend([len(m_dict[u]['es_c']) for u in b])
    en_c_utt_lens.extend([len(m_dict[u]['en_c']) for u in b])
    
        

In [ ]:
plt.hist(es_c_utt_lens, bins=10)

In [ ]:
plt.hist(en_c_utt_lens, bins=10, color='g')

In [ ]:
100 * 10 * 20 / 1000

In [ ]:
batch_data['X'].shape

### Set up encoder

In [ ]:
def check_forward_deep_cnn(h):
    # check and prepare for 2d convolutions
    if CNN_TYPE == DEEP_2D_CNN:
        h = F.expand_dims(h, 2)
        # h = F.reshape(h, (h.shape[:2] + tuple([-1,SPEECH_DIM // 3])))
    h = F.swapaxes(h,1,2)
    print(h.shape)

    for i, cnn_layer in enumerate(model.cnns):
        h = model[cnn_layer](h)
        print(h.shape)
        # h = F.max_pooling_nd(h, ksize=cnn_max_pool[i],
        #                      stride=cnn_max_pool[i],
        #                      pad=max_pool_pad)
        # batch normalization before non-linearity
        if USE_BN:
            bn_lname = '{0:s}_bn'.format(cnn_layer)
            h = model[bn_lname](h)
        h = F.relu(h)
        print(h.shape)

    # out dimension:
    # batch size * num time frames after pooling * cnn out dim
    if CNN_TYPE == DEEP_2D_CNN:
        h = F.swapaxes(h,1,2)
        h = F.reshape(h, h.shape[:2] + tuple([-1]))
        h = F.rollaxis(h,1)
    else:
        h = F.rollaxis(h, 2)
    print(h.shape)

In [ ]:
h = check_forward_deep_cnn(batch_data['X'])

In [ ]:
h = model.forward_deep_cnn(batch_data['X'])

In [ ]:
len(model.cnns)

In [ ]:
h.shape

In [ ]:
model.forward_rnn(h)

In [ ]:
model.forward_enc(batch_data['X'])
model.set_decoder_state()
y = F.swapaxes(batch_data['y'], 0, 1)
# with cmodel.forward_enc(batch_data['X'])
# with chainer.using_config('train', True):
#     myloss = model.decode_batch(y)

In [ ]:
model.enc_states.shape, 1280 /2/3//4

### Start decoding

In [ ]:
model.embed_units*2, model.n_units

In [ ]:
batch_size = y.shape[1]
ht = Variable(xp.zeros((batch_size, model.n_units), dtype=xp.float32))
print(ht.shape)
for curr_word, next_word in zip(y, y[1:]):
    embed_id = model.embed_dec(curr_word)
    print(embed_id.shape, curr_word.shape)
    rnn_in = F.concat((embed_id, ht), axis=1)
    print(rnn_in.shape)
    h = model.feed_rnn(rnn_in, model.rnn_dec)
    cv, _ = model.compute_context_vector(h)
    cv_hdec = F.concat((cv, h), axis=1)
    ht = model.context(cv_hdec)
    # batch normalization before non-linearity
    if USE_BN:
        ht = model.context_bn(ht)
    ht = F.tanh(ht)

    predicted_out = model.out(ht)
    print(predicted_out.shape)
    print(ht.shape)
    break

In [ ]:
ht = xp.zeros((y.shape[1],model.n_units), dtype=xp.float32)

In [ ]:
embed_id.shape

In [ ]:
F.concat((embed_id, ht), axis=1).shape

In [ ]:
model[model.rnn_enc[-1]].h.shape

In [ ]:
model.set_decoder_state(h)

In [ ]:
model.enc_states.shape, h.shape

### Understanding batch matmul and matmul

In [ ]:
weights = F.batch_matmul(model.enc_states, h)

In [ ]:
weights.shape, h.shape

In [ ]:
weights[0,:5]

In [ ]:
model.enc_states[0].shape, h[0].shape

In [ ]:
F.matmul(model.enc_states[0], h[0])[:5]

### Attention!

In [ ]:
ht = model.attn_Wa(h)

In [ ]:
ht.shape

In [ ]:
weights = F.batch_matmul(model.enc_states, ht)

In [ ]:
weights.shape

### Batch and Layer Normalization

In [ ]:
X = batch_data['X']
y = batch_data['y']

In [ ]:
X.shape, y.shape, X[0,:2].shape

In [ ]:
l1 = L.Linear(69,10)
l1.to_gpu()

In [ ]:
h1 = l1(X[0,:2])

In [ ]:
h1.shape

In [ ]:
h1

In [ ]:
bn1 = L.BatchNormalization(10)
bn1.to_gpu()

In [ ]:
ln1 = L.LayerNormalization(10)
ln1.to_gpu()

### Checking BN, LN on network

In [ ]:
h.shape

In [ ]:
# Layer normalization
xp.mean(h.data,axis=1), xp.var(h.data,axis=1)

In [ ]:
xp.mean(model.L1_dec_ln(h).data,axis=1).shape

In [ ]:
xp.mean(h.data,axis=0).shape

In [ ]:
X = batch_data['X']
h = F.swapaxes(X,1,2)

h = F.relu(model[model.cnns[0]](h))
h = F.max_pooling_nd(h, ksize=cnn_max_pool[0],
                                 stride=cnn_max_pool[0],
                                 pad=max_pool_pad)
# h = model.forward_deep_cnn(h)
# print(h.shape)
# h = F.rollaxis(h, 2)
h = F.swapaxes(h,1,2)
print(h.shape)

In [ ]:
huhu = L.BatchNormalization(256)
huhu.to_gpu()

In [ ]:
h[:,0,:].shape

In [ ]:
huhu(h[:,0,:]).shape

In [ ]:
model.CNN_0_bn(h)

In [ ]:
h.shape, model.CNN_2_bn.params

In [ ]:
xp.mean(h.data, axis=(0,1)), xp.var(h.data, axis=(0,1))

In [ ]:
# xp.mean(model.CNN_2_bn(h).data[0], axis=1)

In [ ]:
xp.mean(h[0].data,axis=0)

### mean and var across all inputs in the batch and the same dimension should be 0, and ~1 respectively

The shape should be the number of hidden units

In [ ]:
h2_bn = bn1(h1)

In [ ]:
h2_bn

In [ ]:
xp.mean(h1.data,axis=0), xp.var(h1.data,axis=0)

In [ ]:
xp.mean(h2_bn.data,axis=0), xp.var(h2_bn.data,axis=0)

In [ ]:
h2_ln = ln1(h1)

In [ ]:
h2_ln

### mean and var across all layer activations within a batch should be 0, and ~1 respectively

The shape should be the batch size

In [ ]:
xp.mean(h2_ln.data,axis=1), xp.var(h2_ln.data,axis=1)

### CNN across frequencies

In [ ]:
a = xp.random.randn(39)

In [ ]:
a

In [ ]:
b = F.expand_dims(a, axis=1)

In [ ]:
c = F.reshape(a, (-1,13))

In [ ]:
c.shape

In [ ]:
c.data[1,:5]

In [ ]:
a = xp.load("fbank_out/fisher_train/20050908/20050908_182943_22_fsp-A-1.npy")

In [ ]:
a.shape

In [ ]:
b = F.reshape(a, (1,len(a),-1,13))

In [ ]:
b.shape

In [ ]:
b = F.swapaxes(b,1,2)

In [ ]:
b.shape

In [ ]:
a[0,:20]

In [ ]:
b[0,0,:2,:5]

In [ ]:
c2d_1 = L.Convolution2D(in_channels=None, 
                      out_channels=32, 
                      ksize=(3,3), 
                      stride=(2,2),pad=3//2)

c2d_bn_1 = L.BatchNormalization(32)

c2d_2 = L.Convolution2D(in_channels=None, 
                      out_channels=32, 
                      ksize=(3,3), 
                      stride=(2,2),pad=3//2)

c2d_bn_2 = L.BatchNormalization(32)

c2d_1.to_gpu()
c2d_2.to_gpu()
c2d_bn_1.to_gpu()
c2d_bn_2.to_gpu()

In [ ]:
X = batch_data['X']
y = batch_data['y']
X.shape, y.shape

In [ ]:
h = F.reshape(X, (X.shape[:2] + tuple([-1,SPEECH_DIM // 3])))
h.shape

In [ ]:
h = F.swapaxes(h,1,2)
h.shape

In [ ]:
c = c2d_1(h)

In [ ]:
c2d_1.W.shape

In [ ]:
c.shape

In [ ]:
c_bn = c2d_bn_1(c)

In [ ]:
c_bn.shape

In [ ]:
d = c2d_2(c)

In [ ]:
c2d_2.W.shape

In [ ]:
d.shape

In [ ]:
d_bn = c2d_bn_1(d)

In [ ]:
d_bn.shape

In [ ]:
e = F.swapaxes(d, 1,2)

In [ ]:
e.shape

In [ ]:
e.shape[:2] + tuple([-1])

In [ ]:
e = F.reshape(e, e.shape[:2] + tuple([-1]))

In [ ]:
e.shape

In [ ]:
cnd_1 = L.ConvolutionND(ndim=3,
                        in_channels=1, 
                        out_channels=32, 
                        ksize=(3,3,3), 
                        stride=2,pad=3//2)

cnd_2 = L.ConvolutionND(ndim=3,
                        in_channels=1, 
                        out_channels=32, 
                        ksize=(3,3,3), 
                        stride=2,pad=3//2)

cnd_1.to_gpu()
cnd_2.to_gpu()

### NStepBiGRU

In [ ]:
ng_enc = L.NStepGRU(2,SPEECH_DIM, hidden_units, DROPOUT_RATIO)
ng_dec = L.NStepGRU(2,hidden_units, hidden_units, DROPOUT_RATIO)
ng_enc.to_gpu()
ng_dec.to_gpu()

In [ ]:
X, y = batch_data['X'][:,:20,:], batch_data['y']

In [ ]:
X.shape, y.shape

In [ ]:
X_list = list(X)

In [ ]:
h,o = ng_enc(hx=None, xs=X_list)

In [ ]:
X_list[0].shape

In [ ]:
h.shape, len(o)

In [ ]:
h[:1].shape,h[-1,:,:].shape

In [ ]:
o[1]

In [ ]:
Variable(o)

In [ ]:
o = Variable(np.asarray(o))
o.shape

In [ ]:
hd,od = ng_dec(hx=h,xs=o)

In [ ]:
hd.shape

### Make predictions

In [ ]:
pred_sents, loss = feed_model(map_dict[key],
                              b_dict=bucket_dict[key],
                              vocab_dict=vocab_dict,
                              batch_size=64,
                              x_key=enc_key,
                              y_key=dec_key,
                              train=True,
                              cat_speech_path=os.path.join(out_path, key))

In [ ]:
[(w.decode(),f) for w,f in vocab_dict['es_c']['freq'].items()]

In [ ]:
"".join([w.decode() for w in map_dict['fisher_train']['20050908_182943_22_fsp-B-7']['es_c']])

In [ ]:
xp = cuda.cupy if gpuid >= 0 else np

In [ ]:
print(len(vocab_dict[enc_key]['w2i']))
print(len(vocab_dict[dec_key]['w2i']))

In [ ]:
if gpuid >= 0:
    # print("here")
    cuda.get_device_from_id(gpuid).use()

In [ ]:
model = SpeechEncoderDecoder()

if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [ ]:
if OPTIMIZER_ADAM1_SGD_0:
    print("using ADAM optimizer")
    # optimizer = optimizers.Adam(alpha=0.001,
    #                             beta1=0.9,
    #                             beta2=0.999,
    #                             eps=1e-08)
    # optimizer = optimizers.AdaGrad()
    optimizer = optimizers.Adam()
    optimizer.setup(model)
else:
    print("using SGD optimizer")
    optimizer = optimizers.SGD(lr=0.01)
    optimizer.setup(model)

if WEIGHT_DECAY:
    optimizer.add_hook(chainer.optimizer.WeightDecay(WD_RATIO))

# gradient clipping
optimizer.add_hook(chainer.optimizer.GradientClipping(threshold=2))

In [ ]:
print(type(map_dict['fisher_train']['20050908_182943_22_fsp-B-7']['en_w']))
print(type(map_dict['fisher_dev']['20051009_182032_217_fsp-B-1']['en_w'][0]))

In [ ]:
map_dict['fisher_dev']['20051009_182032_217_fsp-B-1']['en_w'][0]

In [ ]:
feed_model(map_dict['fisher_train'], b_dict=bucket_dict['fisher_train'], 
           vocab_dict=vocab_dict, batch_size=BATCH_SIZE, 
           x_key=enc_key, y_key=dec_key, cat_speech_path=out_path)

In [ ]:
vocab_dict = pickle.load(open("out/train_vocab.dict", "rb"))
vocab_dict['es_w']['w2i'][b'solas']

In [ ]:
map_dict['fisher_train']['20050908_182943_22_fsp-B-7']['es_w']

In [ ]:
len(pred_sents), pred_sents[0]

In [ ]:
for b in b_shuffled:
    bucket = bucket_dict['fisher_train']['buckets'][b]
    b_len = len(bucket)
    print(b_len)
    for i in range(0,b_len, 32):
        

## Noise simulation test

In [ ]:
list(map_dict['fisher_train'].keys())[:5]

In [ ]:
test_batch = buckets_dict['train'][0]['X_fwd'][:10]

In [ ]:
test_batch.shape

In [ ]:
noise_var = np.full(test_batch.shape, 0.01, dtype=np.float32)

In [ ]:
noise = Variable(np.random.normal(0.0, .01, size=test_batch.shape).astype(xp.float32))
noise.to_gpu()

In [ ]:
test_batch_noise = test_batch + noise

In [ ]:
test_batch[0,0,:5], test_batch_noise[0,0,:5]

## Checking for consistency in train/dev sets

In [ ]:
print(set([f.split(".")[0] for f in text_data['train'].keys()]), sep=", ")

In [ ]:
print(set([f.split(".")[0] for f in text_data['dev'].keys()]), sep=", ")

In [ ]:
train_fids = []
for b in buckets_dict["train"]:
    train_fids.extend(b['f'])
dev_fids = []
for b in buckets_dict["dev"]:
    dev_fids.extend(b['f'])
print(len(train_fids), len(dev_fids))
print(set([f.split(".")[0] for f in train_fids]), sep=", ")
print(set([f.split(".")[0] for f in dev_fids]), sep=", ")

In [ ]:
text_data['train'].keys()

## Predictions

In [ ]:
pr_ids, en_ids = predict(4,2, cat="dev", display=True)

In [ ]:
model_fil

In [ ]:
model_fil_old = 'small/seq2seq_sen-4000_hwy4-dec2_emb-512-h-512__word_gru_sspkr_drpt-0_noise-0.20_l2-0.001_cnn-num100-range9-199-20-pool900_10.model'

In [ ]:
!ls $model_fil_old

In [ ]:
serializers.load_npz(model_fil_old, model)

In [ ]:
pplx_new = compute_pplx(cat="dev", num_sent=100)

In [ ]:
pr_ids, en_ids = predict(3,2, cat="train", display=True)

In [ ]:
pr_ids, en_ids = predict(3,2, cat="dev", display=True)

### Create wavs for dev, dev2 set

In [ ]:
map_dict['fisher_dev']['20051025_212334_337_fsp-B-39']['seg']

In [ ]:
list_of_segments = list(map_dict['fisher_dev'].keys())

In [ ]:
len(list_of_segments)

In [ ]:
'20051025_212334_337_fsp-B-39'.rsplit('-',1)

In [ ]:
list_of_segments[0].rsplit('-',1)

In [ ]:
set_of_segments = set([l.rsplit('-',1)[0] for l in list_of_segments])

In [ ]:
len(set_of_segments)

In [ ]:
sph2pipe_path = '''/disk/scratch/s1444673/zero/installs/kaldi/egs/fisher_callhome_spanish/s5/../../../tools/sph2pipe_v2.5/sph2pipe'''

In [ ]:
import subprocess

In [ ]:
with open("callhome_fbank_out/wav.scp", "r") as wavs_f, open("callhome_fbank_out/wavs/dev_wavs.scp", "w") as dev_wavs_f:
    for line in wavs_f:
        for seg in set_of_segments:
            if seg in line:
#                 print(seg)
                out_line =line.split(" ",1)[1].replace(' |', " > {0:s}.wav".format(seg))
                dev_wavs_f.write(out_line)
#                 subprocess.call([sph2pipe_path, "-f", "wav", "-p", "-c", "1", ])

#         if any(x in line for x in set_of_segments):
#             print(line.split(" ",1)[1])
#             dev_wavs_f.write(line)

In [ ]:
import scipy.io.wavfile
from IPython.display import Audio
from IPython.display import display

In [ ]:
wav_path = "callhome_fbank_out/wavs/20051009_182032_217_fsp-A.wav"

In [ ]:
haha = scipy.io.wavfile.read(wav_path)

In [ ]:
haha[1]

In [ ]:
map_dict['fisher_dev']['20051009_182032_217_fsp-B-1']

In [ ]:
Audio(haha[1][:20000], rate=8000)

In [ ]:
nn = np.load("callhome_fbank_out/wavs/20051009_182032_217_fsp-A.wav")